In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import os
import random
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import maskrcnn_resnet50_fpn
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [13]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.6 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [14]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [15]:
data_yaml_path = "/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/data.yaml"  # points to your custom dataset yaml
weights_path = "yolov8n-seg.pt"  # Use pretrained YOLOv8 segmentation weights


In [16]:
# Load model
model = YOLO(weights_path)

100%|██████████| 6.74M/6.74M [00:00<00:00, 91.9MB/s]


In [ ]:
# Train the model on your dataset
model.train(data=data_yaml_path, epochs=5, imgsz=640, task="segment")

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

100%|██████████| 755k/755k [00:00<00:00, 21.2MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/labels... 1109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1109/1109 [00:04<00:00, 270.76it/s]

train: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/images/IMG_3647_block_1_2_png_jpg.rf.0796e002c87e60355b14a1a7e36e6128.jpg: 1 duplicate labels removed
train: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/images/IMG_3647_block_1_2_png_jpg.rf.2122f7adcbe77d67d0a237c89bd8ea81.jpg: 1 duplicate labels removed
train: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/images/IMG_3647_block_1_2_png_jpg.rf.3b1455b12ea1db0a62f22a332cd2d29f.jpg: 1 duplicate labels removed
train: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/images/IMG_3647_block_1_2_png_jpg.rf.6462084eeebf7c80dbd01f48c88cc7d7.jpg: 1 duplicate labels removed
train: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train/images/IMG_3647_block_1_2_png_jp

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.2±0.1 ms, read: 16.1±3.6 MB/s, size: 59.8 KB)


val: Scanning /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels... 191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 191/191 [00:00<00:00, 286.14it/s]

val: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/images/IMG_3743_block_1_0_png_jpg.rf.543766cd632a2699c6fa14ad5851adbb.jpg: 1 duplicate labels removed
val: /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/images/IMG_3743_block_1_1_png_jpg.rf.22a2e72846091adcd22c79d11c717244.jpg: 3 duplicate labels removed
WARNING ⚠️ val: Cache directory /kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid is not writeable, cache not saved.


Plotting labels to runs/segment/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/train
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      2.068      3.217      3.252      1.246        589        640:  81%|████████▏ | 57/70 [12:58<02:52, 13.27s/it]